<a href="https://colab.research.google.com/github/namyaagrawal03/Insurance_fraud/blob/main/interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade lida
!pip install --upgrade fastapi
!pip install --upgrade pydantic
!pip install --upgrade pydantic-core
!pip install --upgrade tensorflow-probability

In [ ]:
!pip install typing-extensions==3.10.0.2

  Using cached typing_extensions-3.10.0.2-py3-none-any.whl (26 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sqlalchemy 2.0.23 requires typing-extensions>=4.2.0, but you have typing-extensions 3.10.0.2 which is incompatible.
arviz 0.15.1 requires typing-extensions>=4.1.0, but you have typing-extensions 3.10.0.2 which is incompatible.
chex 0.1.7 requires typing-extensions>=4.2.0; python_version < "3.11", but you have typing-extensions 3.10.0.2 which is incompatible.
fastapi 0.108.0 requires typing-extensions>=4.8.0, but you have typing-extensions 3.10.0.2 which is incompatible.
flax 0.7.5 requires typing-extensions>=4.2, but you have typing-extensions 3.10.0.2 whi

In [ ]:
!pip install gradio -q

In [ ]:
import gradio as gr

In [ ]:
import joblib
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from google.colab import drive
drive.mount('/content/drive')
rfc = joblib.load('/content/drive/MyDrive/Colab Notebooks/trained_model.pkl')
scaler = joblib.load('/content/drive/MyDrive/Colab Notebooks/scaler_filename.pkl')
preprocessor = joblib.load('/content/drive/MyDrive/Colab Notebooks/preprocessor.joblib')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def predict_fraud(Make, AccidentArea, Sex, Age, MaritalStatus, PolicyType,
       VehiclePrice, Deductible, DriverRating, PastNumberOfClaims,
       AgeOfVehicle, PoliceReportFiled, WitnessPresent, AgentType,
       NumberOfCars):

    # Create test data as a DataFrame
    test_data = pd.DataFrame({
        'Make': [Make],
        'AccidentArea': [AccidentArea],
        'Sex': [Sex],
        'Age': [Age],
        'MaritalStatus': [MaritalStatus],
        'PolicyType': [PolicyType],
        'VehiclePrice': [VehiclePrice],
        'Deductible': [Deductible],
        'DriverRating': [DriverRating],
        'PastNumberOfClaims': [PastNumberOfClaims],
        'AgeOfVehicle': [AgeOfVehicle],
        'PoliceReportFiled': [PoliceReportFiled],
        'WitnessPresent': [WitnessPresent],
        'AgentType': [AgentType],
        'NumberOfCars': [NumberOfCars]
    })
    # Assuming test_data is your DataFrame with categorical columns
    categorical_cols = test_data[['Make', 'AccidentArea', 'Sex', 'MaritalStatus', 'PolicyType',
                                'VehiclePrice', 'PastNumberOfClaims', 'AgeOfVehicle',
                                'PoliceReportFiled', 'WitnessPresent', 'AgentType', 'NumberOfCars']]

# Assuming you have new_data which has similar categorical columns
    X_processed = preprocessor.transform(categorical_cols)
    ohe_column_names = preprocessor.named_transformers_['cat']['encoder'].get_feature_names_out(categorical_cols.columns)

# Convert the sparse matrix to a DataFrame with column names
    X_processed = pd.DataFrame.sparse.from_spmatrix(X_processed, columns=ohe_column_names)


# Now, new_data_encoded has the same one-hot encoding as the loaded_categorical_cols

    numerical_cols = test_data[['Age', 'Deductible', 'DriverRating']]

    # Select numerical columns
    x = pd.concat([numerical_cols.reset_index(drop=True), X_processed.reset_index(drop=True)], axis=1)

    x[[ 'Age', 'Deductible',  'DriverRating']]=scaler.transform(x[['Age', 'Deductible',  'DriverRating']])


    # Combine numerical and categorical columns

    predicted_label = rfc.predict(x)

    if predicted_label[0] == 1:
        output_label = "Fraud Detected"
    else:
        output_label = "No Fraud Detected"
    return output_label

In [ ]:
import gradio as gr

# Define the dropdowns
dropdowns = [
    gr.Dropdown(
        ['Pontiac', 'Toyota', 'Saturn', 'Ferrari', 'Jaguar', 'VW', 'Lexus', 'Honda', 'Ford', 'Accura', 'BMW', 'Mazda', 'Nisson', 'Chevrolet', 'Mercury', 'Porche', 'Mecedes', 'Saab', 'Dodge'], label="Make of the Vehicle"
    ),
    gr.Dropdown(
        ['Urban', 'Rural'], label="Accident Area"
    ),
    gr.Dropdown(
        ['Male', 'Female'], label="Sex"
    ),
    gr.Dropdown(
        ['16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37',
         '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59','60', '61',
         '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78','79', '80'], label="Age"
    ),
    gr.Dropdown(
        ['Single',  'Married', 'Divorced', 'Widow'], label="Marital Status"
    ),
    gr.Dropdown(
        ['Sedan - Liability', 'Sport - Collision', 'Sport - All Perils', 'Utility - All Perils', 'Sport - Liability', 'Utility - Liability',
         'Sedan - All Perils', 'Utility - Collision', 'Sedan - Collision'], label="Policy Type"
    ),
    gr.Dropdown(
        ['less than 20000', '20000 to 29000' ,'30000 to 39000',  '40000 to 59000',  '60000 to 69000',  'more than 69000' ], label="Price of the Vehicle"
    ),
     gr.Dropdown(
        ['300' ,'400', '500', '700'], label="Deductible Amount", info="Select the closest amount"
    ),
    gr.Dropdown(
        ['1', '2', '3', '4'], label="Driver Rating"
    ),
     gr.Dropdown(
        [ 'none', '1', '2 to 4', 'more than 4'], label="Past Number of Claims"
    ),
     gr.Dropdown(
        ['new',  '2 years',  '3 years',  '4 years', '5 years',  '6 years',  '7 years', 'more than 7'], label="Age of the Vehicle"
    ),
     gr.Dropdown(
        [ 'Yes', 'No'], label="Was a police report filed?"
    ),
    gr.Dropdown(
        [ 'Yes', 'No'], label="Was there any witness present?"
    ),
    gr.Dropdown(
        ['Internal', 'External'], label="Agent Type"
    ),
    gr.Dropdown(
        ['1 vehicle','2 vehicles',  '3 to 4', '5 to 8', 'more than 8'], label="Number of cars owned"
    )

]

Gradio interface with custom layout
interface = gr.Interface(
    fn=predict_fraud,  # Replace with your prediction function
    inputs=dropdowns,
    outputs="text",

)
interface.launch(debug=True)



Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://2e27a03b8ff3b08f37.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 489, in call_prediction
    output = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 232, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1561, in process_api
 